<h2> Do not run all the cells if you already have the files</h2>

<h3> Importing libraries</h3>


In [2]:
from os import listdir
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from pickle import load

<h3>Extract the features from each photo in our directory</h3>

In [8]:
def extract_features(directory):
    # load the model
    model = VGG16()
    # modify the model
    model.layers.pop()
    "We dont need the last layer of  prediction layer"
    model = Model(inputs = model.inputs, outputs = model.layers[-1].output)
    print(model.summary())
    # Extracting features from each photo
    features = dict()
    for name in listdir(directory):
        # load an image 
        filename = directory + '/' + name
        image = load_img(filename, target_size(224,224))
        # convert the image pixels to a numpy array
        image = img_to_array(iamge)
        # reshape the data for the model
        image = image.reshape((1,image.shape[0], image.shape[1], image.shape[2]))
        # prepare the image for the VGG model
        image = preprocess_input(image)
        # get the features
        features = model.predict(image, verbose = 0)
        # get image id
        image_id = name.split('.')[0]
        # store features
        features[image_id] = feature
        # showing the file names
        print('--> %s ' % name)
    return features

# extract features from all images
directory = 'YOUR DIRECTORY'
features = extract_features(directory)
print('extracted features len: %d' % len(features))
# Save those to a file
dump(features, open('features.pkl', 'wb'))    

<h3>Preparing the Text Data</h3>

In [ ]:
# Load the doc to memory
def load_doc(filename):
    # open the file as read only mode
    file = open(filename, 'r')
    # read all the texts
    text = file.read()
    # Close the file
    file.Close()
    return text

filename = "YOUR FILE NAME"

# load descriptions
doc = load_doc(filename)

In [ ]:
# Extract description for images
def load_descriptions(doc):
    mapping = dict()
    # Process lines
    for line in doc.split('\n'):
        # Split the line by whitespace
        tokens = line.split()
        if (len(line) < 2):
            continue
        # Take the first toekn as the image id, the rest as the description
        image_id, image_desc = tokens[0], tokens[1:]
        # Remove filename from image id
        image_id = image_id.split('.')[0]
        # Convert description tokens back to string
        image_desc = ' '.join(image_desc)
        # create the list if needed
        if image_id not in mapping:
            mapping[image_id] = list()
        # store description
        mapping[image_id].append(image_desc)
    return mapping

#Parse descriptinos
descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))
    

<h3>Next we need to clean the description text.</h3> <br>
<p>Convert all the words to lowercase<br>
Remove all punctuation<br>
Remove all the words that are one character or less in length like 'a' <br>
Remove all the words with numbers in them<br></p>

In [ ]:
import string

def clean_descriptions(descriptions):
    # Prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for key, desc_list in descriptions.items():
        for i in range(len(desc_list)):
            desc = desc_list[i]
            #tokenize
            desc = desc.split()
            #Convert to lower case
            desc = [word.lower() for word in desc]
            #Remove punctuation from each token
            desc = [w.translate(table) for w in desc]
            # Remove hanging 's' and 'a' 
            desc = [word for word in desc if len(word) > 1]
            # Remove tokens with numbers in them
            desc = [word for word in desc if word.isalpha()]
            # Store as string
            desc_list[i] = ' '.join(desc)
    
# Clean description
clean_descriptions(descriptions)

In [ ]:
# Lets find out the size of our vocabulary
# means lets convert the loaded discriptions into a vocabulary of words
def to_vocabulary(descriptions):
    # Build a list of all desctiption strings
    all_desc = set()
    for key in descriptions.keys():
        [all_desc.update(d.split()) for d in descriptions[key]]
    return all_desc

# Summarizing the vocabulary
vocabulary = to_vocabulary(descriptions)
print('Vocavulary Size : %d' % len(vocabulary))

<h3>!! Do not run this cell if you already have the descriptions.txt file !! </h3>
<p> It will take time to generate 'descriptions.txt' file</p>

In [ ]:
# Saving our filtered image id and descriptions in a file
def save_descriptions(descriptions, filename):
    counter = 1
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + ' ' + desc)
        data = '\n'.join(lines)
        print(lines)
        file = open(filename, 'w')
        file.write(data)
        file.close()
# Saving descriptions
save_descriptions(descriptions, 'descriptions.txt')

<h2> Developing the Deep Learning Model </h2>

In [4]:
from numpy import array
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input,Dense, LSTM, Embedding, Dropout,GRU
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


<h3> Loading the data</h3>

In [ ]:
# Load doc to memory

def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all the texts 
    text = file.read()
    #Close the file
    file.close()
    return text

# Load a pre-defined list of photo identifiers
def load_set(filename):
    doc = load_doc(filename)
    dataset = list()
    # Process line by line
    for line in doc.split('\n'):
        # Skip empty lines
        if (len(line) < 2):
            continue
        # Get image identifier
        identifier = line.split('.')[0]
        dataset.append(identifier)
    return set(dataset)

In [ ]:
# get the clean descriptions from our dataset
def load_clean_descriptions(filename, dataset):
    # Load the document
    doc = load_doc(filename)
    descriptions = dict()
    for line in doc.split('\n'):
        # Split line by white space
        tokens = line.split()
        # Split id from desctiptions just like before
        image_id, image_desc = tokens[0], tokens[1:]
        # Skip images not in the set
        if image_id in dataset:
            # Create list
            if image_id not in descriptions:
                descriptions[image_id] = list()
            # Wrap descriptions in tokens
            desc = 'startseq' + ' '.join(image_desc) + ' endseq'
            # Store 
            descriptions[image_id].append(desc)
    return descriptions


In [ ]:
# Load photo features
def load_photo_features(filename, dataset):
    # loading features
    all_features = load(open(filename, 'rb'))
    features = {k: all_features[k] for k in dataset}
    return features

In [ ]:
# Load training dataset

filename= 'D:\Ongoing work\PROJECTS\Image Captioning\Flickr_8k.trainImages.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))
# Descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: Train=%d' % len(train_descriptions))
# photo features
train_features = load_photo_features('features.pkl', train)
print('photos: train = %d ' %len(train_features))


In [ ]:
# Convert a dictionary of clean descriptions to a list of descriptions
from keras.preprocessing.text import Tokenizer

def to_lines(descriptions):
    all_desc = list()
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

# Fit a tokenizer given caption descriptions 
def create_tokenizer(descriptions):
    lines = to_lines(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

# Prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size %d' % vocab_size)

In [ ]:
# get the lenght of the description having max len
def max_length(descriptions):
    lines = to_lines(descriptions)
    return max(len(d.split()) for d in lines)

max_length = max_length(train_descriptions)
print(max_length)

<h4> Using data Generator for loading data to model</h4>

In [ ]:
def data_generator(descriptions, photos, tokenizer, max_length):
    # Loop over all images
    while 1:
        for key, desc_list in descriptions.items():
            # getting the photo features
            photo = photos[key][0]
            in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc_list, photo)
            yield [[in_img, in_seq], out_word]            
            

In [ ]:
# Creating sequences of image, input sequences and output words for an image
def create_sequences(tokenizer, max_length, desc_list, photo):
    X1, X2, y = list(), list(), list()
    # walk through each description for the image
    for desc in desc_list:
        # Encode the sequence
        seq = tokenizer.texts_to_sequences([desc])[0]
        # Split one sequence into multiple X, y pairs
        for i in range(1, len(seq)):
            # Split into output and input pair
            in_seq, out_seq = seq[:i], seq[i]
            # Pad input sequence
            in_seq = pad_sequences([in_seq], maxlen = max_length)[0]
            # encode output sequenc
            out_seq = to_categorical([out_seq], num_classes = vocab_size)[0]
            # Store
            X1.append(photo)
            X2.append(in_seq)
            y.append(out_seq)
    return array(X1), array(X2), array(y)
            
    

In [ ]:
# Testing the data generator
generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
inputs, outputs = next(generator)
print(inputs[0].shape)
print(inputs[1].shape)
print(outputs.shape)

<h3> Defining the captioning model</h3>

In [2]:
def define_model(vocab_size, max_length):
    # feature extractor model
    inputs1 = Input(shape=(4096,))
    fe1 = Dropout(0.5)(Inputs1)
    fe2 = Dense(256, activation = 'relu')(fe1)
    # sequence model
    inputs2 = Input(shape=(max_length))
    se1 = Embedding(vocab_size, 256, mask_zero = True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)
    # Decoder model
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation = 'relu')(decoder1)
    outputs = Dense(vocab_size, activation= 'softmax')(decoder2)
    # adding it together [img, seq][word]
    model = Model(inputs=[inputs1, inputs2], outputs= outputs)
    # compile the model
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
    # summarize model
    model.summary()
    return model

In [ ]:
# load training dataset (6K)
filename = 'D:\Ongoing work\PROJECTS\Image Captioning\Flickr_8k.trainImages.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# photo features
train_features = load_photo_features('features.pkl', train)
print('Photos: train=%d' % len(train_features))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
print(max_length)

In [ ]:
# Fit the training model and dump its parameters

# define the model
model = define_model(vocab_size, max_length)
# train the model, run epochs manually and save after each epoch
epochs = 20
steps = len(train_descriptions)
for i in range(epochs):
    # create the data generator
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
    # fit for one epoch
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    # save model
    model.save('model_' + str(i) + '.h5')

<h3>Evaluate the model</h3>

In [8]:
# mapping an integer to a word (opposite of tokenization)
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None


In [ ]:
# generating a description for an image
def generate_desc(model, tokenizer, photo, max_length):
    # seed the generation process
    in_text = 'startseq'
    # iterate over the whole length of sequence
    for i in range(max_length):
        # integer encode input sequence
        sequence = pad_sequences([sequence], maxlen= max_length)
        # predict next word
        yhat = model.predict([photo, sequence], verbose = 0)
        # convert probability to integer
        yhat = argmax(yhat)
        # map integer to word
        word = word_for_id(yhat, tokenizer)
        # stop if cannot map the word
        if word is None:
            break
    return in_text   

<h3>Using BLEU Score to evaluate the model</h3>

In [10]:
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
    actual, predicted = list(), list()
    count = 1 # just to see the progress
    # go through the whole set
    for key, desc_list in description.items():
        print(count)
        # generate descriptions
        yhat = generate_desc(model, tokenizer, photos[key], max_length)
        # store actual and predicted
        original_desc = [d.split() for d in desc_list]
        actual.append(original_desc)
        predicted.append(yhat.split())
        count += 1
    # Calculating the BLUE score
    # Calculating bleu score
    print('BLEU-1 :-> %f ' %corpus_bleu(actual, predicted, weights = (1.0,0,0,0)))
    print('BLEU-2 :-> %f ' %corpus_bleu(actual, predicted, weights = (0.5,0.5,0,0)))
    print('BLEU-3 :-> %f ' %corpus_bleu(actual, predicted, weights = (0.3,0.3,0.3,0)))
    print('BLEU-4 :-> %f ' %corpus_bleu(actual, predicted, weights = (0.25,0.25,0.25,0.25)))

In [ ]:
filename = 'D:\Ongoing work\PROJECTS\Image Captioning\Flickr_8k.testImages.txt'
test = load_set(filename)
small_test = set(itertools.islice(test, 100))
print('Data set : %d' % len(small_test))
# descriptions
test_descriptions = load_clean_descriptions('descriptions.txt', small_test)
print('Descriptions: test= %d' % len(test_descriptions))
# photo features
test_features = load_photo_features('features.pkl', small_test)
print('Photos: test= %d' % len(test_features))


# load the model
filename = 'D:\Ongoing work\PROJECTS\Image Captioning\model_19.h5'

model = load_model(filename)
# evaluate model
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)



In [ ]:
from keras.preprocessing.text import Tokenizer
from pickle import dump

In [ ]:
# Convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
    all_desc = list()
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
    lines = to_lines(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [ ]:
# Loading the training dataset
filename = 'D:\Ongoing work\PROJECTS\Image Captioning\Flickr_8k.trainImages.txt'
train = load_set(filename)
print('Dataset : %d' %len(train))
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('descriptions: train: %d' % len(train_descriptions))
#prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))